In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Uploading kaggle.json file

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ainuraa","key":"7bed69885a22c506d607b1c4ce58e4ff"}'}

Install the kaggle API

In [ ]:
!pip install -q kaggle

Moving the kaggle.json file into ~/.kaggle

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!cp kaggle.json ~/.kaggle/

Downloading files

In [ ]:
!kaggle datasets download -d paramaggarwal/fashion-product-images-dataset -p '/content'

100% 23.1G/23.1G [03:27<00:00, 123MB/s]
100% 23.1G/23.1G [03:27<00:00, 119MB/s]


In [ ]:
!unzip '/content/fashion-product-images-dataset.zip'

Preprocessing

In [ ]:
import os, json
import pandas as pd


path_to_json = '/content/fashion-dataset/styles/'
json_files = [json.load(open(path_to_json+pos_json)) for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

In [ ]:
len(json_files)

44446

id, productDisplayName,  'brandName', 'gender', articleAttributes

In [ ]:
import re


product_descriptions = []
for style in json_files:
    dic = {}
    data = style['data']
    displayCategories = ''
    if 'displayCategories' in data:
      displayCategories = data['displayCategories'].split(',')[0]

    name = data['productDisplayName'].replace(data['brandName'],"")+ ' ' + \
          data['season'] + ' ' + data['usage'] + ' ' + displayCategories + ' '
    if data['baseColour'] not in name:
      name += data['baseColour']
    name = re.sub(r"[.,\/#!$&%\^\*;:{}=\_`~()@0-9\[\]]", '', name)
    name = re.sub(r'\s+', ' ', name).strip()
    dic['id'] = data['id']
    dic['description'] = name
    product_descriptions.append(dic)

In [ ]:
with open('/content/drive/MyDrive/AML/project/Github/Product-image-generation-from-text-description/result.json', 'w') as fp:
    json.dump(product_descriptions, fp)